In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates. All rights reserved.

# Render a textured mesh

This tutorial shows how to:
- load a mesh and textures from an `.obj` file.
- set up a renderer
- render the mesh
- vary the rendering settings such as lighting and camera position
- use the batching features of the pytorch3d API to render the mesh from different viewpoints

## 0. Install and Import modules

Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [2]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("1.13.", "2.0.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 585.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61406 sha256=c70bcc8c65d17f08b84376f5f093ae35b4facb9ba70efb8a3e16b952f7276f36
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=069079f82e5fc7154a82aa18c2017aeaf04373e150f50fd33bb64c09aebeb75d
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu118_pyt201/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 123.3 MB/s eta 0:00:00


In [3]:
import os
import torch
import matplotlib.pyplot as plt

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    DirectionalLights,
    Materials,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    TexturesUV,
    TexturesVertex
)

# add path for demo utils functions
import sys
import os
sys.path.append(os.path.abspath(''))

If using **Google Colab**, fetch the utils file for plotting image grids:

In [4]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
from plot_image_grid import image_grid

--2023-08-09 17:45:56--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘plot_image_grid.py’

plot_image_grid.py  100%[===================>]   1.57K  --.-KB/s    in 0s      

2023-08-09 17:45:56 (31.8 MB/s) - ‘plot_image_grid.py’ saved [1608/1608]



OR if running **locally** uncomment and run the following cell:

In [5]:
# from utils import image_grid

In [6]:
!ls

plot_image_grid.py  __pycache__  sample_data


In [7]:
from google.colab import drive

drive.mount('/content/drive')

import os

os.chdir("/content/drive/MyDrive/Colab Notebooks")


Mounted at /content/drive


In [8]:
!ls

1_b.gif			      rendering_generic_3d_representations.ipynb
2.1.gif			      rendering_point_clouds_from_rgbd.ipynb
3_b.gif			      rendering_texture_obj.ipynb
data			      render_tetrahedron_and_retexturing.ipynb
deform_form.ipynb	      render_texture_mesh_and_dolly_zoom.ipynb
plotly_rendered_meshes.ipynb


In [9]:
if torch.cuda.is_available():
  device=torch.device("cuda:0")
else:
  device=torch.device("cpu")
  print("gpu is not available")


In [10]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("1.13.", "2.0.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [11]:
from torch.cuda import is_available

from pytorch3d.renderer.mesh.textures import TexturesAtlas
import torch

from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import(
    chamfer_distance,
    mesh_edge_loss,
    mesh_normal_consistency,
    mesh_laplacian_smoothing
)

from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    SoftPhongShader,
    MeshRenderer,
    MeshRasterizer,
    RasterizationSettings,
    TexturesAtlas,
    TexturesVertex,
    PointsRenderer,
    AlphaCompositor,
    NormWeightedCompositor,
    PointsRasterizationSettings,
    PointsRenderer,
    PulsarPointsRenderer,
    PointsRasterizer,
)

import numpy as np

from tqdm.notebook import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

In [12]:
!ls

1_b.gif			      rendering_generic_3d_representations.ipynb
2.1.gif			      rendering_point_clouds_from_rgbd.ipynb
3_b.gif			      rendering_texture_obj.ipynb
data			      render_tetrahedron_and_retexturing.ipynb
deform_form.ipynb	      render_texture_mesh_and_dolly_zoom.ipynb
plotly_rendered_meshes.ipynb


In [13]:
import pickle

file_path = 'data/rgbd_data.pkl'

with open(file_path, 'rb') as file:
    data = pickle.load(file)

keys = data.keys()

# Print the keys
print("Keys in the dictionary:")
for key in keys:
    print(key)



Keys in the dictionary:
rgb1
mask1
depth1
rgb2
mask2
depth2
cameras1
cameras2


In [60]:

#we need to load image, mask, depth, camera
image1=data['rgb1']
image2=data['rgb2']
mask1=data['mask1']
mask2=data['mask2']
depth1=data['depth1']
depth2=data['depth2']
camera1=data['cameras1']
camera2=data['cameras2']

assert image1.shape[0] == image1.shape[1]
print(image1.shape)

(800, 800, 3)


In [91]:
#this is for image1

image_shape=image1.shape[0]
ndc_pixel_coordinate= torch.linspace(1,-1, image_shape)
#dc has size [800]
Y,X=torch.meshgrid(ndc_pixel_coordinate,ndc_pixel_coordinate)
X=X.to(device)
Y=Y.to(device)
#X and Y have [800,800]


depth1.shape
depth1=torch.tensor(depth1).to(device)
xy_depth=torch.stack([X,Y,depth1], dim=2)
xy_depth.shape
camera1=camera1.to(device)
#xy_depth have shape [800,800,3]
points1=camera1.unproject_points(
    xy_depth.to(device), in_ndc=False, from_ndc=False, world_coordiante=True
)


#torch.Size([800, 800, 3])

points1=points1[mask1>0.5]

print(image1.shape)
print(mask1.shape)
rgb1=image1[mask1>0.5]

# print(type(rgb1))
rgb1=torch.tensor(rgb1).to(device)
alpha=torch.ones_like(rgb1)[...,:1]

#rgb1 has shape torch.Size([125035, 3])
#alpha has shape torch.Size([125035, 1])
rgb1=torch.cat([rgb1,alpha],dim=1)




(800, 800, 3)
(800, 800)


<ipython-input-91-751d523a7d19>:13: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [90]:
#this is for image2

image_shape2=image2.shape[0]
ndc_pixel_coordinate2= torch.linspace(1,-1, image_shape)
#dc has size [800]
Y2,X2=torch.meshgrid(ndc_pixel_coordinate2,ndc_pixel_coordinate2)
X2=X2.to(device)
Y2=Y2.to(device)
#X and Y have [800,800]


depth2.shape
depth2=torch.tensor(depth2).to(device)
xy_depth2=torch.stack([X2,Y2,depth2], dim=2)
xy_depth2.shape
camera2=camera2.to(device)
#xy_depth have shape [800,800,3]
points2=camera2.unproject_points(
    xy_depth2.to(device), in_ndc=False, from_ndc=False, world_coordiante=True
)


#torch.Size([800, 800, 3])

points2=points2[mask2>0.5]


rgb2=image2[mask2>0.5]

# print(type(rgb1))
rgb2=torch.tensor(rgb2).to(device)
alpha2=torch.ones_like(rgb2)[...,:1]

#rgb1 has shape torch.Size([125035, 3])
#alpha has shape torch.Size([125035, 1])
rgb2=torch.cat([rgb2,alpha2],dim=1)




<ipython-input-90-087dbaf628d9>:13: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [92]:
#now we have points1, rgb1, points2, rgb2

#for the 1st image,


point_cloud1=Pointclouds(points=[points1], features=[rgb1])
point_cloud2=Pointclouds(points=[points2], features=[rgb2])

points3, rgb3= torch.cat([points1, points2], dim=0), torch.cat([rgb1,rgb2],dim=0)

point_cloud3=Pointclouds(points=[points3],features=[rgb3])



In [93]:
fig = plot_scene({
    "subplot1": {
        "plant": point_cloud3
    }
})
fig.show()

In [101]:

from PIL import Image, ImageDraw
import imageio

def render_360_pc(point_cloud, image_size=256, output_path='images/q_5-1_pc1.gif',
 num_views=12, fps=15, elev=10, dist=7, device=None, background_color=(1, 1, 1), rotate_R=False):

    renderer = get_points_renderer(image_size=image_size, background_color=background_color)

    angles = np.linspace(-180, 180, num_views, endpoint=False)
    images = []
    for i in range(num_views):
        R, T = pytorch3d.renderer.look_at_view_transform(
        dist=dist,
        elev=elev,
        azim=angles[i],
    )
        # rotate upside down
        if rotate_R:
            R = pytorch3d.transforms.euler_angles_to_matrix(torch.Tensor([0, 0, np.pi]), "XYZ") @ R

        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=R,
        T=T,
        device=device
    )

        rend = renderer(point_cloud, cameras=cameras)
        rend = rend[0, ..., :3].cpu().numpy()

        image = Image.fromarray((rend * 255).astype(np.uint8))
        draw = ImageDraw.Draw(image)
        draw.text((20, 20), f"angle: {angles[i]:.0f}", fill=(255, 0, 0))
        images.append(np.array(image))
    imageio.mimsave(output_path, images, fps=fps)

In [99]:
def get_points_renderer(
    image_size=512, device=None, radius=0.01, background_color=(1, 1, 1)
):
    """
    Returns a Pytorch3D renderer for point clouds.

    Args:
        image_size (int): The rendered image size.
        device (torch.device): The torch device to use (CPU or GPU). If not specified,
            will automatically use GPU if available, otherwise CPU.
        radius (float): The radius of the rendered point in NDC.
        background_color (tuple): The background color of the rendered image.

    Returns:
        PointsRenderer.
    """
    if device is None:
        if torch.cuda.is_available():
            device = torch.device("cuda:0")
        else:
            device = torch.device("cpu")
    raster_settings = PointsRasterizationSettings(image_size=image_size, radius=radius,)
    renderer = PointsRenderer(
        rasterizer=PointsRasterizer(raster_settings=raster_settings),
        compositor=AlphaCompositor(background_color=background_color),
    )
    return renderer

In [103]:
image_size=512
background_color=(1, 1, 1)

render_360_pc(point_cloud1, image_size=image_size, output_path='q_5-1_pc1.gif', fps=10, device=device, background_color=background_color, rotate_R=True)
render_360_pc(point_cloud2, image_size=image_size, output_path='q_5-1_pc2.gif', fps=10, device=device, background_color=background_color, rotate_R=True)
render_360_pc(point_cloud3, image_size=image_size, output_path='q_5-1_pc_union.gif', fps=10, device=device, background_color=background_color, rotate_R=True)
